In [1]:
##check for gpu
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-6aaead0e-a78e-5441-a207-af28790655c8)


##Get helper Functions



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import serires of funtions 
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2021-08-12 05:13:42--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-12 05:13:42 (69.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



##Get a text dataset

link :https://www.kaggle.com/c/nlp-getting-started


In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-12 05:13:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-08-12 05:13:44 (129 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##Visulaizing the dataset

In [4]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#shuffle training dataframe
train_df_shuffled =train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
#test dataframe look
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#count the examples
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(test_df),len(train_df)

(3263, 7613)

In [9]:
#visulalize random samples
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
   _,text,target=row
   print(f"Target :{target}","(real disater)" if target > 0 else "(not real disater)")
   print(f"Text :\n {text}\n")
   print("---\n")

Target :0 (not real disater)
Text :
 .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: .: RT DrAyesha4: #IndiaKoMunTorJawabDo

Indian Army kiÛ_ http://t.co/WJLJq3yA4g

---

Target :0 (not real disater)
Text :
 @Demolition_d best grill u fkn pleb

---

Target :0 (not real disater)
Text :
 Well done Celtic Fingers crossed for Aberdeen tomorrow night! 
Armageddon eh.... ??

---

Target :0 (not real disater)
Text :
 New print available on http://t.co/ucy5fEA9yu! - 'Waiting Too Long' by Pamela Blizzard - http://t.co/dnwwo1YbRK

---

Target :1 (real disater)
Text :
 Reasons I should have gone to Warped today: tony played issues showed up sleeping w sirens played attila is there issues issues issues

---



##Split data into train and Validation

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
#use train test split 
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [12]:
#check the length
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

##Converting Text into numbers

when dealing with text problem ,

1. builld the model to convert the words into numbers
  
  *Tokanization
  
  *embedding



##Tokenization

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [15]:
#use the defult text vectorization parameters
text_vectorizer =TextVectorization(max_tokens=None,#how many words taken into token
                                   standardize="lower_and_strip_punctuation",
                                   split="whitespace",
                                   ngrams=None,
                                   output_mode="int",
                                   output_sequence_length=None,
                                   pad_to_max_tokens=True)

In [16]:
text_vectorizer

In [17]:
#find the avarge number of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
#setup the text vectorization varibales
max_vocab_length=10000 #max number in vocabulary
max_length=15 #max length our sequences 

text_vectorizer =TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)



In [19]:
#fit the text vectorizer to train data
text_vectorizer.adapt(train_sentences)

In [20]:
#create a sample sentence and tokenize it
sample_sentence ="there is fire in my town"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,  42,   4,  13, 801,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
#choose random sentence from the training daatset
random_sentence=random.choice(train_sentences)
print(f"Orignal text :\n {random_sentence} \
      \n\nVectorized version:")

text_vectorizer([random_sentence])

Orignal text :
 @flickershowell oh wow my heart collapsed cool im crying cool cool       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  392,  970,   13,  435,  439,  607,   32, 1693,  607,  607,
           0,    0,    0,    0]])>

In [22]:
#get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() #get all the unique words in train data
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[:-5]

print(f"Number of words in vocab :{len(words_in_vocab)}")
print(f"5 most common words in vocab :{top_5_words}")
print(f"5 least common of words in vocab :{bottom_5_words}")

Number of words in vocab :10000
5 most common words in vocab :['', '[UNK]', 'the', 'a', 'in']
5 least common of words in vocab :['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'yo

##Create a Embedding using Embedding Layer

PARAMETERS:

1.INPUT DIM-size of the vocabulary

2.OUTPUT DIM -size of the output embedding vector

3.INPUT LENGTH-length of the sequence being passed to the embedding layer

In [23]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

embedding

In [24]:
random_sentence=random.choice(train_sentences)
print(f"Orignal Text :\n{random_sentence}\
      \n\nEmbedded Vesrion :")

#embed the setence
sample_embed =embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal Text :
#360WiseNews : China's Stock Market Crash: Are There Gems In The Rubble? http://t.co/eaTFro3d5x      

Embedded Vesrion :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00306028, -0.04635401,  0.01948898, ...,  0.03113688,
          0.00164989,  0.01257786],
        [-0.00743042, -0.01863489,  0.02846419, ..., -0.02157953,
         -0.02228817, -0.0275092 ],
        [-0.04135608, -0.03197344, -0.02314085, ...,  0.04489893,
          0.00769712,  0.037487  ],
        ...,
        [-0.0392121 ,  0.04415897, -0.02652125, ..., -0.02110068,
         -0.00717859,  0.02542815],
        [-0.0392121 ,  0.04415897, -0.02652125, ..., -0.02110068,
         -0.00717859,  0.02542815],
        [-0.0392121 ,  0.04415897, -0.02652125, ..., -0.02110068,
         -0.00717859,  0.02542815]]], dtype=float32)>

In [25]:
#check out single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00306028, -0.04635401,  0.01948898, -0.02937733,  0.01710348,
        -0.03327179, -0.03956999,  0.03026769,  0.00836641, -0.04014628,
        -0.02703266,  0.02264923, -0.04857378, -0.04598159,  0.01662341,
        -0.02189693, -0.03918825,  0.01622838,  0.00471538,  0.02681491,
         0.00011977, -0.03718366,  0.00609775,  0.01059967, -0.01728709,
         0.00486418,  0.01438048, -0.0203041 , -0.02834832,  0.04543724,
         0.04534039,  0.01202495,  0.02001638,  0.02343505,  0.03442669,
         0.02205758,  0.04712443, -0.04675436, -0.01901701,  0.02347875,
         0.01832349, -0.01426551,  0.0480369 , -0.02036873, -0.01206031,
         0.04786453,  0.04783026,  0.03043714, -0.01372176,  0.00038403,
        -0.03475202,  0.02507284, -0.01174617,  0.04179807, -0.02531575,
         0.00408892,  0.01374092,  0.00796164,  0.00305356,  0.00744613,
        -0.00342461,  0.00227239, -0.00720656, -0.00994643,  0.01585872,
  

##Modeling a Text Dataset and Runnning Experiments

model 0-naive bayes(baseline)

model 1-Feed forward neaural Network(dense model)

model 2-LSTM Model

model 3-GRU Model

model 4-Bidirectional Model

model 5-ID Convolutional Neural Network Model

model 6-Tensorflow hub pre trained feature extractor Model

model 7-same model with 10% training data Model

*steps:

+create the model

+compile the model

+fit the model

+evaluate the model


##Model 0 -Getting a Baseline model

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokanization and modeling pipeline
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clf",MultinomialNB())
])

#fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
#evaluate the baseline model
baseline_score=model_0.score(val_sentences,val_labels)
print(f"model arcive score :{baseline_score*100:.2f}")
baseline_score

model arcive score :79.27


0.7926509186351706

In [28]:
#make predictions
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

##Create evaluation Function

Accuracy

precition

recall

f1-score


In [29]:
#funtion to evaluate
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calcualte_results(y_true,y_preds):
  """
  Calculates model accuracy,preciton,recall and f1 score of a binary classification model
  """

  #calcualte model accuracy
  model_accuracy=accuracy_score(y_true,y_preds)*100

  #calculate model prestion,recall,f1-score 
  model_precision,model_recall,model_f1,_ =precision_recall_fscore_support(y_true,y_preds,average="weighted")
  model_results={"accuracy":model_accuracy,
                 "precision":model_precision,
                 "recall":model_recall,
                 "f1":model_f1}

  return model_results



In [30]:
#get baseline results
baseline_results =calcualte_results(y_true=val_labels,
                                    y_preds=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 1 -Build a Dense Model

In [31]:
#crate a tensorboard callbacks
from helper_functions import create_tensorboard_callback

#create directory to save logs
SAVE_DIR="model_logs"

In [32]:
#build model with functional API
from tensorflow.keras import layers

inputs =layers.Input(shape=(1,),dtype =tf.string)

#turn text into numbers
x=text_vectorizer(inputs)

#create an embedding of the numberized inputs
x=embedding(x)

x=layers.GlobalAveragePooling1D()(x)

outputs=layers.Dense(1,activation="sigmoid")(x)

model_1=tf.keras.Model(inputs,outputs,name="model_1")

In [33]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
#fit the model
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])


Saving TensorBoard log files to: model_logs/model_1_dense/20210812-051353
Epoch 1/5
215/215 [==============================] - 7s 17ms/step - loss: 0.6122 - accuracy: 0.6872 - val_loss: 0.5371 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4432 - accuracy: 0.8196 - val_loss: 0.4735 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3478 - accuracy: 0.8608 - val_loss: 0.4569 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2854 - accuracy: 0.8910 - val_loss: 0.4730 - val_accuracy: 0.7966
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2382 - accuracy: 0.9092 - val_loss: 0.4779 - val_accuracy: 0.7848


In [36]:
#check the results
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4779 - accuracy: 0.7848


[0.4779472351074219, 0.7847769260406494]

In [37]:
#make predictions
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs

array([[4.15931076e-01],
       [7.90129185e-01],
       [9.97939765e-01],
       [1.05873391e-01],
       [1.29555672e-01],
       [9.23762619e-01],
       [9.28484619e-01],
       [9.93400753e-01],
       [9.66484964e-01],
       [2.43699029e-01],
       [1.22866429e-01],
       [7.04367220e-01],
       [5.17876334e-02],
       [1.78502232e-01],
       [4.32325900e-03],
       [1.37676567e-01],
       [2.78427321e-02],
       [8.44475403e-02],
       [2.12128311e-01],
       [5.43041110e-01],
       [9.09006476e-01],
       [4.56785187e-02],
       [4.19946879e-01],
       [7.63333216e-02],
       [9.52937603e-01],
       [9.99140024e-01],
       [3.48372646e-02],
       [6.20371662e-02],
       [3.06064636e-02],
       [2.35899612e-01],
       [5.34323514e-01],
       [2.57323086e-01],
       [5.39276540e-01],
       [2.00260043e-01],
       [4.91854727e-01],
       [4.81834710e-02],
       [9.93504107e-01],
       [1.30970731e-01],
       [3.42145786e-02],
       [9.98869956e-01],


In [38]:
#convert predicions to label format
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [39]:
#calculte model_1_results
model_1_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_1_preds)

model_1_results

{'accuracy': 78.4776902887139,
 'f1': 0.781600538849599,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391}

In [40]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Visializing Laerned Embeddings

In [41]:
#get the vocabulary from the text vectorization
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
#Model 1 summary
model_1.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [43]:
#get the weight metrics of embedding layer
embed_weights =model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [44]:
embed_weights

array([[-0.02536139,  0.06096428, -0.0119827 , ..., -0.00505189,
        -0.02499432,  0.01199919],
       [-0.02448511, -0.019349  ,  0.04980431, ..., -0.00600509,
        -0.02528718,  0.00455572],
       [-0.02506931,  0.00533979, -0.01589601, ..., -0.00872235,
         0.01499189, -0.05043396],
       ...,
       [ 0.0113014 ,  0.0138986 ,  0.00542418, ...,  0.02469906,
         0.03284517, -0.03183302],
       [ 0.05673734,  0.0840546 ,  0.0444994 , ...,  0.0225273 ,
        -0.05553868, -0.03306964],
       [ 0.0404105 ,  0.04319906,  0.10830651, ...,  0.0816893 ,
        -0.08811206, -0.03317143]], dtype=float32)

In [45]:
#create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [46]:
#downlaod files from colab to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Model 2-Recurrent Neural Network

Structure of the RNN:

Input-->Tokenize-->Embedding-->Layers-->Output

In [47]:
#cerate an lstm model
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)

# x=layers.LSTM(units=64,return_sequences=True)(x)
# print(x.shape)

x=layers.LSTM(64)(x)
print(x.shape)

# x=layers.Dense(64,activation="relu")(x)
# print(x.shape)

outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [48]:
#get the summary
model_2.summary()


Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [49]:
#compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [50]:
#fit the model
model_2_history =model_2.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210812-051417
Epoch 1/5
215/215 [==============================] - 10s 21ms/step - loss: 0.2238 - accuracy: 0.9155 - val_loss: 0.5626 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1587 - accuracy: 0.9404 - val_loss: 0.6249 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1290 - accuracy: 0.9523 - val_loss: 0.6011 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1050 - accuracy: 0.9622 - val_loss: 0.8874 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0845 - accuracy: 0.9657 - val_loss: 0.8467 - val_accuracy: 0.7769


In [51]:
#make predictions 
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.6314127e-01],
       [7.4891633e-01],
       [9.9942929e-01],
       [8.8022500e-02],
       [3.2451999e-04],
       [9.9791080e-01],
       [7.6017392e-01],
       [9.9968302e-01],
       [9.9949038e-01],
       [5.4605937e-01]], dtype=float32)

In [52]:
#convert model 2 pred to labels
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
#calculate the model results
model_2_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_2_preds)

model_2_results

{'accuracy': 77.69028871391076,
 'f1': 0.7756776970544694,
 'precision': 0.7772431692902212,
 'recall': 0.7769028871391076}

##Model 3 -GRU -Gated Recurrent Unit



In [54]:
#build rnn using the gru unit
from tensorflow.keras import layers
inputs =layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)

#when stack recurrent layers ,return_sequences must be "True"
x=layers.GRU(64)(x)
# x=layers.LSTM(64,return_sequences=True)(x)
# x=layers.GRU(64,return_sequences=True)(x)
# x=layers.Dense(64,activation="relu")(x)
# x=layers.GlobalAveragePooling1D()(x)
outputs =layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")



In [55]:
#Model Summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [56]:
model_3.summary()


Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [57]:

#compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
#fit the model
model_3_history =model_3.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20210812-051441
Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.1524 - accuracy: 0.9396 - val_loss: 0.7372 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0836 - accuracy: 0.9688 - val_loss: 0.7837 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0691 - accuracy: 0.9730 - val_loss: 0.9035 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0568 - accuracy: 0.9761 - val_loss: 1.2813 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0526 - accuracy: 0.9772 - val_loss: 1.3029 - val_accuracy: 0.7730


In [59]:
#make predictions 
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[2.3758951e-04],
       [7.8544384e-01],
       [9.9993038e-01],
       [6.6212051e-02],
       [4.4161054e-05],
       [9.9979562e-01],
       [9.5265353e-01],
       [9.9996412e-01],
       [9.9993789e-01],
       [9.8432124e-01]], dtype=float32)

In [60]:
#convert model 2 pred to labels
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [61]:
#calculate the model results
model_3_results=calcualte_results(y_true=val_labels,
                                y_preds=model_3_preds)
model_3_results


{'accuracy': 77.29658792650919,
 'f1': 0.7686844166386851,
 'precision': 0.7802583549165839,
 'recall': 0.7729658792650919}

In [62]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 4 -Bi directional RNN

In [63]:
#build the bidirectinal layer
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)

# x=layers.Bidirectional(layers.LSTM(32,return_sequences=True))(x)
# print(x.shape)

x=layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)

outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

(None, 15, 128)
(None, 128)


In [64]:
#model summary
model_4.summary()


Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [65]:
#compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
#fit the model
model_4_history =model_4.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "Bidirectional")])

Saving TensorBoard log files to: model_logs/Bidirectional/20210812-051504
Epoch 1/5
215/215 [==============================] - 8s 25ms/step - loss: 0.1087 - accuracy: 0.9675 - val_loss: 0.9866 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0531 - accuracy: 0.9764 - val_loss: 1.1281 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0478 - accuracy: 0.9790 - val_loss: 1.1940 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0440 - accuracy: 0.9803 - val_loss: 1.2699 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0393 - accuracy: 0.9810 - val_loss: 1.4649 - val_accuracy: 0.7651


In [67]:
#make predictions 
model_4_pred_probs=model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.1394802e-03],
       [7.8733462e-01],
       [9.9997652e-01],
       [1.9833426e-01],
       [1.8723174e-05],
       [9.9983680e-01],
       [9.7531772e-01],
       [9.9998558e-01],
       [9.9997735e-01],
       [9.9912304e-01]], dtype=float32)

In [68]:
#convert model 2 pred to labels
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
#calculate the model results
model_4_results=calcualte_results(y_true=val_labels,
                                y_preds=model_4_preds)
model_4_results


{'accuracy': 76.50918635170603,
 'f1': 0.7637474572934756,
 'precision': 0.765325307490926,
 'recall': 0.7650918635170604}

In [70]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [71]:
model_3_results

{'accuracy': 77.29658792650919,
 'f1': 0.7686844166386851,
 'precision': 0.7802583549165839,
 'recall': 0.7729658792650919}

##Model 5-Convolutional Neural Networks for Text(Other types of sequence data)

In [72]:
#test out embedding ,conv1D and max pooling
from tensorflow.keras import layers
embedding_test=embedding(text_vectorizer(["this is a sentence"]))
conv_1d =layers.Conv1D(filters=32,
                       kernel_size=5,
                       activation="relu",
                       padding="valid")

conv_1d_output=conv_1d(embedding_test)

max_pool=layers.GlobalMaxPool1D()
max_pool_output=max_pool(conv_1d_output)


embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [73]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.00000000e+00, 1.86860608e-03, 0.00000000e+00, 5.80779165e-02,
         7.75218755e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         3.10726501e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 1.00450069e-02, 0.00000000e+00, 0.00000000e+00,
         1.64223760e-02, 0.00000000e+00, 2.02605743e-02, 6.90704584e-02,
         0.00000000e+00, 0.00000000e+00, 3.23277637e-02, 3.55823115e-02,
         1.84371807e-02, 0.00000000e+00, 5.07204235e-02, 5.17691346e-03,
         6.70939013e-02, 0.00000000e+00, 0.00000000e+00, 1.15419134e-01],
        [2.28678212e-02, 7.08778109e-03, 5.09592297e-04, 5.18251061e-02,
         6.12445734e-02, 3.73967849e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 9.38127749e-03, 0.00000000e+00,
         6.10838011e-02, 0.00000000e+00, 0.00000000e+00, 2.60848533e-0

In [79]:
#create 1 dimentional convolutional layer to model sequemces
from tensorflow.keras import layers
inputs =layers.Input(shape=(1,),dtype ="string")
x=text_vectorizer(inputs)
x=embedding(x)
x =layers.Conv1D(filters=64,kernel_size=5,activation="relu",padding="valid",strides=1 )(x)
x=layers.GlobalMaxPool1D()(x)
outputs =layers.Dense(1,activation="sigmoid")(x)
model_5 =tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

#complete conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#model summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [80]:
#fit the model
model_5_history=model_5.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                                    "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210812-052540
Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.1265 - accuracy: 0.9581 - val_loss: 0.8463 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0765 - accuracy: 0.9720 - val_loss: 1.0077 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0605 - accuracy: 0.9761 - val_loss: 1.0827 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0555 - accuracy: 0.9780 - val_loss: 1.1663 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0513 - accuracy: 0.9791 - val_loss: 1.2010 - val_accuracy: 0.7559


In [81]:
#make predictions
model_5_pred_probs=model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[8.3538443e-02],
       [9.0456581e-01],
       [9.9979168e-01],
       [4.7805317e-02],
       [2.1956494e-08],
       [9.9772781e-01],
       [9.9011022e-01],
       [9.9999523e-01],
       [9.9999869e-01],
       [8.7411654e-01]], dtype=float32)

In [82]:
#convert preds to labels
model_5_preds=tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [84]:
model_5_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_5_preds)

model_5_results

{'accuracy': 75.59055118110236,
 'f1': 0.75433369124314,
 'precision': 0.7562023016353725,
 'recall': 0.7559055118110236}

##Model 6 -Tenserflow Hub Pre trained Sentence Encoder

In [160]:
import tensorflow_hub as hub
MODEL_URL="https://tfhub.dev/google/universal-sentence-encoder/4"
embed =hub.load(MODEL_URL)
embed_samples=embed([sample_sentence,
                     "Universal sentences encoder turns sentence into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[ 0.00722868 -0.005796    0.0202273  -0.00420904  0.06025884 -0.00071852
  0.01552324  0.01020351  0.01834386 -0.07381546  0.0846421  -0.01530199
  0.08684347  0.07090341  0.04933254  0.00370435  0.02794769 -0.0105089
  0.07017351 -0.05877548 -0.03603147 -0.08475685  0.02922119 -0.00414703
 -0.08365953 -0.01280118  0.01175286  0.01066282 -0.04120266 -0.01146041
  0.0717511  -0.08176148  0.00437718 -0.02967084 -0.04301972 -0.06009932
  0.03883352 -0.07603729  0.03248987 -0.00633733 -0.05175756  0.00262169
  0.01169271  0.03034221 -0.09188599  0.01105333 -0.03462776 -0.03979238
  0.05136561 -0.05436561], shape=(50,), dtype=float32)


In [161]:
#create keras layer using the use layer from tensorflow hub
sentence_encoder_layer=hub.KerasLayer(MODEL_URL,
                                      input_shape=[],
                                      dtype=tf.string,
                                      trainable=False,
                                      name="USE")

In [162]:
#Create model using sequentinal api
model_6=tf.keras.Sequential([
                             sentence_encoder_layer,
                             layers.Dense(64,activation="relu"),
                             layers.Dense(1,activation="sigmoid",name="output_layer"),
],name="Model_6_USE")

In [163]:
#Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])



In [164]:
#model summary
model_6.summary()

Model: "Model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_13 (Dense)             (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [165]:
#trained the classfier on use layer
model_6_history=model_6.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                   "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20210812-073719
Epoch 1/5
215/215 [==============================] - 7s 24ms/step - loss: 0.5081 - accuracy: 0.7786 - val_loss: 0.4484 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4142 - accuracy: 0.8175 - val_loss: 0.4390 - val_accuracy: 0.8045
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4003 - accuracy: 0.8234 - val_loss: 0.4316 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3920 - accuracy: 0.8248 - val_loss: 0.4288 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3841 - accuracy: 0.8298 - val_loss: 0.4256 - val_accuracy: 0.8123


In [166]:
#make predcitons
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.18011288],
       [0.7915415 ],
       [0.990482  ],
       [0.20290871],
       [0.71211475],
       [0.72752887],
       [0.9825935 ],
       [0.97815835],
       [0.9341666 ],
       [0.10417556]], dtype=float32)

In [167]:
#convert preds to labels
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [168]:
model_6_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_6_preds)

model_6_results

{'accuracy': 81.23359580052494,
 'f1': 0.8108895606331169,
 'precision': 0.8142769487815527,
 'recall': 0.8123359580052494}

In [169]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 7 -Tensorflow HUB with 10% training data

always keep eye on the datasplits to avoid data leak,Otherwise evaluation metrics can be unacceptable.

In [177]:
 # #create subsets of 10 trainind data
# train_10_percent =train_df_shuffled[["text","target"]].sample(frac=0.1,random_state=42)
# train_sentences_10_percent=train_10_percent["text"].to_list()
# train_labels_10_percent=train_10_percent["target"].to_list()

# len(train_sentences_10_percent),len(train_labels_10_percent)


In [178]:
#making better data spits
train_10_precent_split=int(0.1*len(train_sentences))
train_sentences_10_percent=train_sentences[:train_10_precent_split]
train_labels_10_percent=train_labels[:train_10_precent_split]
len(train_sentences_10_percent)

685

In [179]:
train_labels_10_percent

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,

In [180]:
import numpy as np

pd.Series(np.array(train_labels_10_percent)).value_counts()


0    406
1    279
dtype: int64

In [181]:
len(train_df_shuffled)

7613

In [182]:
# #check the traget
# train_10_percent["target"].value_counts()

In [183]:
#Create model using sequentinal api for 10 percen data
model_7=tf.keras.Sequential([
                             sentence_encoder_layer,
                             layers.Dense(64,activation="relu"),
                             layers.Dense(1,activation="sigmoid",name="output_layer"),
],name="Model_7_USE_10_percent_data")

In [184]:
#Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [185]:
#model summary
model_7.summary()

Model: "Model_7_USE_10_percent_data"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_14 (Dense)             (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [186]:
#trained the classfier on use layer
model_7_history=model_7.fit(train_sentences_10_percent,
                            train_labels_10_percent,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                   "tf_hub_sentence_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent_correct_split/20210812-073804
Epoch 1/5
22/22 [==============================] - 3s 92ms/step - loss: 0.6720 - accuracy: 0.6672 - val_loss: 0.6507 - val_accuracy: 0.7454
Epoch 2/5
22/22 [==============================] - 0s 21ms/step - loss: 0.6062 - accuracy: 0.8102 - val_loss: 0.5951 - val_accuracy: 0.7677
Epoch 3/5
22/22 [==============================] - 1s 25ms/step - loss: 0.5295 - accuracy: 0.8175 - val_loss: 0.5377 - val_accuracy: 0.7677
Epoch 4/5
22/22 [==============================] - 0s 21ms/step - loss: 0.4673 - accuracy: 0.8292 - val_loss: 0.5036 - val_accuracy: 0.7743
Epoch 5/5
22/22 [==============================] - 1s 25ms/step - loss: 0.4233 - accuracy: 0.8292 - val_loss: 0.4882 - val_accuracy: 0.7782


In [187]:
#make predcitons
model_7_pred_probs=model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.21878652],
       [0.57279915],
       [0.9193635 ],
       [0.38547847],
       [0.5659331 ],
       [0.7062926 ],
       [0.8951642 ],
       [0.80226916],
       [0.8479768 ],
       [0.17680396]], dtype=float32)

In [188]:
#convert preds to labels
model_7_preds=tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [189]:
model_7_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_7_preds)

model_7_results

{'accuracy': 77.82152230971128,
 'f1': 0.7769459233664658,
 'precision': 0.7786516618308618,
 'recall': 0.7782152230971129}

In [190]:
model_6_results

{'accuracy': 81.23359580052494,
 'f1': 0.8108895606331169,
 'precision': 0.8142769487815527,
 'recall': 0.8123359580052494}

In [191]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}